# 1. Loading the Required Libraries

In [1]:
import numpy as np
import pandas as pd
import string
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings # supress warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
import nltk

# 2. Loading the data

In [2]:
df1 =pd.read_json('train_spider.json')
df2 =pd.read_json('dev.json')
df3 =pd.read_json('train_others.json')

# 3. Summary of the Three dataset

In [3]:
df1.info() # contains 7000 entries of 140 databases


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                7000 non-null   object
 1   query                7000 non-null   object
 2   query_toks           7000 non-null   object
 3   query_toks_no_value  7000 non-null   object
 4   question             7000 non-null   object
 5   question_toks        7000 non-null   object
 6   sql                  7000 non-null   object
dtypes: object(7)
memory usage: 382.9+ KB


In [4]:
df2.info() # contains 1034 entries of 20 databases

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                1034 non-null   object
 1   query                1034 non-null   object
 2   query_toks           1034 non-null   object
 3   query_toks_no_value  1034 non-null   object
 4   question             1034 non-null   object
 5   question_toks        1034 non-null   object
 6   sql                  1034 non-null   object
dtypes: object(7)
memory usage: 56.7+ KB


In [5]:
df3.info() # contains 1659 entries of 6 databases

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1659 entries, 0 to 1658
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                1659 non-null   object
 1   query                1659 non-null   object
 2   query_toks           1659 non-null   object
 3   query_toks_no_value  1659 non-null   object
 4   question             1659 non-null   object
 5   question_toks        1659 non-null   object
 6   sql                  1659 non-null   object
dtypes: object(7)
memory usage: 90.9+ KB


In [6]:
df1.db_id.value_counts()

college_2                          170
college_1                          164
hr_1                               124
store_1                            112
soccer_2                           106
                                  ... 
solvency_ii                        15 
customers_and_products_contacts    15 
soccer_1                           14 
local_govt_mdm                     14 
company_1                          7  
Name: db_id, Length: 140, dtype: int64

In [8]:
df2.db_id.value_counts()

world_1                         120
car_1                           92 
cre_Doc_Template_Mgt            84 
dog_kennels                     82 
flight_2                        80 
student_transcripts_tracking    78 
tvshow                          62 
wta_1                           62 
network_1                       56 
concert_singer                  45 
pets_1                          42 
poker_player                    40 
orchestra                       40 
employee_hire_evaluation        38 
singer                          30 
course_teach                    30 
museum_visit                    18 
battle_death                    16 
voter_1                         15 
real_estate_properties          4  
Name: db_id, dtype: int64

In [10]:
df3['db_id'].value_counts()

scholar        569
geo            564
academic       181
restaurants    125
yelp           111
imdb           109
Name: db_id, dtype: int64

In [11]:
#inspecting the dataset

df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, age, >, 56]","[select, count, (, *, ), from, head, where, age, >, value]",How many heads of the departments are older than 56 ?,"[How, many, heads, of, the, departments, are, older, than, 56, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
1,department_management,"SELECT name , born_state , age FROM head ORDER BY age","[SELECT, name, ,, born_state, ,, age, FROM, head, ORDER, BY, age]","[select, name, ,, born_state, ,, age, from, head, order, by, age]","List the name, born state and age of the heads of departments ordered by age.","[List, the, name, ,, born, state, and, age, of, the, heads, of, departments, ordered, by, age, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 8, False], None]], [0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['asc', [[0, [0, 10, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
2,department_management,"SELECT creation , name , budget_in_billions FROM department","[SELECT, creation, ,, name, ,, budget_in_billions, FROM, department]","[select, creation, ,, name, ,, budget_in_billions, from, department]","List the creation year, name and budget of each department.","[List, the, creation, year, ,, name, and, budget, of, each, department, .]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[0, [0, [0, 3, False], None]], [0, [0, [0, 2, False], None]], [0, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
3,department_management,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department","[SELECT, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), FROM, department]","[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]",What are the maximum and minimum budget of the departments?,"[What, are, the, maximum, and, minimum, budget, of, the, departments, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[1, [0, [0, 5, False], None]], [2, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
4,department_management,SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,"[SELECT, avg, (, num_employees, ), FROM, department, WHERE, ranking, BETWEEN, 10, AND, 15]","[select, avg, (, num_employees, ), from, department, where, ranking, between, value, and, value]",What is the average number of employees of the departments whose rank is between 10 and 15?,"[What, is, the, average, number, of, employees, of, the, departments, whose, rank, is, between, 10, and, 15, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[5, [0, [0, 6, False], None]]]], 'where': [[False, 1, [0, [0, 4, False], None], 10.0, 15.0]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"


In [12]:
df2.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""France""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"


In [13]:
df3.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
1,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, population]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
2,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}"
3,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, c

# We will consider train_spider.json for training,dev.json for validation,and train_others for test purpose.

# 4. Data Pre-processing

Following are the major pre-processing steps used in NLP

•	Removing punctuations e.g ( ) * % @ ! $ .  ,
•	Removing Whitespaces
•	Lower casing
•	Removing URLs
•	Removing Stop words
•	Tokenization
•	Stemming
•	Lemmatization

We need to perform the above mentioned pre-processing steps based on the dataset.

In [14]:
# 4.1 Punctuations Removal
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

df1["clean_question"] = df1['question'].apply(remove_punctuations)
df2["clean_question"] = df2['question'].apply(remove_punctuations)
df3["clean_question"] = df3['question'].apply(remove_punctuations)

In [15]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, age, >, 56]","[select, count, (, *, ), from, head, where, age, >, value]",How many heads of the departments are older than 56 ?,"[How, many, heads, of, the, departments, are, older, than, 56, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many heads of the departments are older than 56
1,department_management,"SELECT name , born_state , age FROM head ORDER BY age","[SELECT, name, ,, born_state, ,, age, FROM, head, ORDER, BY, age]","[select, name, ,, born_state, ,, age, from, head, order, by, age]","List the name, born state and age of the heads of departments ordered by age.","[List, the, name, ,, born, state, and, age, of, the, heads, of, departments, ordered, by, age, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 8, False], None]], [0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['asc', [[0, [0, 10, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the name born state and age of the heads of departments ordered by age
2,department_management,"SELECT creation , name , budget_in_billions FROM department","[SELECT, creation, ,, name, ,, budget_in_billions, FROM, department]","[select, creation, ,, name, ,, budget_in_billions, from, department]","List the creation year, name and budget of each department.","[List, the, creation, year, ,, name, and, budget, of, each, department, .]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[0, [0, [0, 3, False], None]], [0, [0, [0, 2, False], None]], [0, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the creation year name and budget of each department
3,department_management,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department","[SELECT, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), FROM, department]","[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]",What are the maximum and minimum budget of the departments?,"[What, are, the, maximum, and, minimum, budget, of, the, departments, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[1, [0, [0, 5, False], None]], [2, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the maximum and minimum budget of the departments
4,department_management,SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,"[SELECT, avg, (, num_employees, ), FROM, department, WHERE, ranking, BETWEEN, 10, AND, 15]","[select, avg, (, num_employees, ), from, department, where, ranking, between, value, and, value]",What is the average number of employees of the departments whose rank is between 10 and 15?,"[What, is, the, average, number, of, employees, of, the, departments, whose, rank, is, between, 10, and, 15, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[5, [0, [0, 6, False], None]]]], 'where': [[False, 1, [0, [0, 4, False], None], 10.0, 15.0]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the average number of employees of the departments whose rank is between 10 and 15


In [16]:
df2.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""France""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the average minimum and maximum age of all singers from France


In [17]:
df3.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question
0,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what is the biggest city in wyoming
1,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, population]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what wyoming city has the largest population
2,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what is the largest city in wyoming
3,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming

In [18]:
# 4.2 Whitespace Removal
df1['clean_query'] = df1['query'].str.strip(string.whitespace)
df1['clean_question'] = df1['clean_question'].str.strip(string.whitespace)

df2['clean_query'] = df2['query'].str.strip(string.whitespace)
df2['clean_question'] = df2['clean_question'].str.strip(string.whitespace)

df3['clean_query'] = df3['query'].str.strip(string.whitespace)
df3['clean_question'] = df3['clean_question'].str.strip(string.whitespace)

In [19]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, age, >, 56]","[select, count, (, *, ), from, head, where, age, >, value]",How many heads of the departments are older than 56 ?,"[How, many, heads, of, the, departments, are, older, than, 56, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many heads of the departments are older than 56,SELECT count(*) FROM head WHERE age > 56
1,department_management,"SELECT name , born_state , age FROM head ORDER BY age","[SELECT, name, ,, born_state, ,, age, FROM, head, ORDER, BY, age]","[select, name, ,, born_state, ,, age, from, head, order, by, age]","List the name, born state and age of the heads of departments ordered by age.","[List, the, name, ,, born, state, and, age, of, the, heads, of, departments, ordered, by, age, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 8, False], None]], [0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['asc', [[0, [0, 10, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the name born state and age of the heads of departments ordered by age,"SELECT name , born_state , age FROM head ORDER BY age"
2,department_management,"SELECT creation , name , budget_in_billions FROM department","[SELECT, creation, ,, name, ,, budget_in_billions, FROM, department]","[select, creation, ,, name, ,, budget_in_billions, from, department]","List the creation year, name and budget of each department.","[List, the, creation, year, ,, name, and, budget, of, each, department, .]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[0, [0, [0, 3, False], None]], [0, [0, [0, 2, False], None]], [0, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the creation year name and budget of each department,"SELECT creation , name , budget_in_billions FROM department"
3,department_management,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department","[SELECT, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), FROM, department]","[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]",What are the maximum and minimum budget of the departments?,"[What, are, the, maximum, and, minimum, budget, of, the, departments, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[1, [0, [0, 5, False], None]], [2, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the maximum and minimum budget of the departments,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department"
4,department_management,SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,"[SELECT, avg, (, num_employees, ), FROM, department, WHERE, ranking, BETWEEN, 10, AND, 15]","[select, avg, (, num_employees, ), from, department, where, ranking, between, value, and, value]",What is the average number of employees of the departments whose rank is between 10 and 15?,"[What, is, the, average, number, of, employees, of, the, departments, whose, rank, is, between, 10, and, 15, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[5, [0, [0, 6, False], None]]]], 'where': [[False, 1, [0, [0, 4, False], None], 10.0, 15.0]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, '

In [20]:
df2.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have,SELECT count(*) FROM singer
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers,SELECT count(*) FROM singer
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest,"SELECT name , country , age FROM singer ORDER BY age DESC"
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age,"SELECT name , country , age FROM singer ORDER BY age DESC"
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""France""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the average minimum and maximum age of all singers from France,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'"


In [21]:
df3.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query
0,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what is the biggest city in wyoming,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";"
1,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, population]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what wyoming city has the largest population,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";"
2,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [

In [22]:
# 4.3 Lowering the text
df1['query_lower']= df1['clean_query'].apply(lambda x: x.lower())
df1['quest_lower']= df1['clean_question'].apply(lambda x: x.lower())

df2['query_lower']= df2['clean_query'].apply(lambda x: x.lower())
df2['quest_lower']= df2['clean_question'].apply(lambda x: x.lower())

df3['query_lower']= df3['clean_query'].apply(lambda x: x.lower())
df3['quest_lower']= df3['clean_question'].apply(lambda x: x.lower())


In [26]:
df3.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query,query_lower,quest_lower
0,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what is the biggest city in wyoming,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","select city_name from city where population = ( select max ( population ) from city where state_name = ""wyoming"" ) and state_name = ""wyoming"";",what is the biggest city in wyoming
1,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, population]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what wyoming city has the largest population,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","select city_name from city where population = ( select max ( population ) from city where state_name = ""wyoming"" ) and state_name = ""wyoming"";",what wyoming city has the largest population
2,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [

In [27]:
# 4.4 Tokenization
# This step splits the text into smaller units.

from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenization1(text):
    tokens = word_tokenize(text)
    return tokens


#applying function to the column
df1['query_tokenised']= df1['query_lower'].apply(lambda x: tokenization1(x))
df1['quest_tokenised']= df1['quest_lower'].apply(lambda x: tokenization1(x))


df2['query_tokenised']= df2['query_lower'].apply(lambda x: tokenization1(x))
df2['quest_tokenised']= df2['quest_lower'].apply(lambda x: tokenization1(x))

df3['query_tokenised']= df3['query_lower'].apply(lambda x: tokenization1(x))
df3['quest_tokenised']= df3['quest_lower'].apply(lambda x: tokenization1(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query,query_lower,quest_lower,query_tokenised,quest_tokenised
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, age, >, 56]","[select, count, (, *, ), from, head, where, age, >, value]",How many heads of the departments are older than 56 ?,"[How, many, heads, of, the, departments, are, older, than, 56, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many heads of the departments are older than 56,SELECT count(*) FROM head WHERE age > 56,select count(*) from head where age > 56,how many heads of the departments are older than 56,"[select, count, (, *, ), from, head, where, age, >, 56]","[how, many, heads, of, the, departments, are, older, than, 56]"
1,department_management,"SELECT name , born_state , age FROM head ORDER BY age","[SELECT, name, ,, born_state, ,, age, FROM, head, ORDER, BY, age]","[select, name, ,, born_state, ,, age, from, head, order, by, age]","List the name, born state and age of the heads of departments ordered by age.","[List, the, name, ,, born, state, and, age, of, the, heads, of, departments, ordered, by, age, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 8, False], None]], [0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['asc', [[0, [0, 10, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the name born state and age of the heads of departments ordered by age,"SELECT name , born_state , age FROM head ORDER BY age","select name , born_state , age from head order by age",list the name born state and age of the heads of departments ordered by age,"[select, name, ,, born_state, ,, age, from, head, order, by, age]","[list, the, name, born, state, and, age, of, the, heads, of, departments, ordered, by, age]"
2,department_management,"SELECT creation , name , budget_in_billions FROM department","[SELECT, creation, ,, name, ,, budget_in_billions, FROM, department]","[select, creation, ,, name, ,, budget_in_billions, from, department]","List the creation year, name and budget of each department.","[List, the, creation, year, ,, name, and, budget, of, each, department, .]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[0, [0, [0, 3, False], None]], [0, [0, [0, 2, False], None]], [0, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",List the creation year name and budget of each department,"SELECT creation , name , budget_in_billions FROM department","select creation , name , budget_in_billions from department",list the creation year name and budget of each department,"[select, creation, ,, name, ,, budget_in_billions, from, department]","[list, the, creation, year, name, and, budget, of, each, department]"
3,department_management,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department","[SELECT, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), FROM, department]","[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]",What are the maximum and minimum budget of the departments?,"[What, are, the, maximum, and, minimum, budget, of, the, departments, ?]","{'from': {'table_units': [['table_unit', 0]], 'conds': []}, 'select': [False, [[1, [0, [0, 5, False], None]], [2, [0, [0, 5, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the maximum and minimum budget of the departme

In [29]:
# 4.6 Lemmatization : We will use Lemmatization instead of stemming as stemming causes loss of word meaning .Lemmatization
# on the other hand don't lose the meaning of the words in the root form

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df1['quest_lemmatized']=df1['quest_tokenised'].apply(lambda x:lemmatizer(x))

df2['quest_lemmatized']=df2['quest_tokenised'].apply(lambda x:lemmatizer(x))

df3['quest_lemmatized']=df3['quest_tokenised'].apply(lambda x:lemmatizer(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
df3.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query,query_lower,quest_lower,query_tokenised,quest_tokenised,quest_lemmatized
0,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what is the biggest city in wyoming,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","select city_name from city where population = ( select max ( population ) from city where state_name = ""wyoming"" ) and state_name = ""wyoming"";",what is the biggest city in wyoming,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[what, is, the, biggest, city, in, wyoming]","[what, is, the, biggest, city, in, wyoming]"
1,geo,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","[SELECT, city_name, FROM, city, WHERE, population, =, (, SELECT, MAX, (, population, ), FROM, city, WHERE, state_name, =, ``, wyoming, '', ), AND, state_name, =, ``, wyoming, '', ;]","[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, value, ), and, state_name, =, value]",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, population]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 7, False], None]]]], 'where': [[False, 2, [0, [0, 8, False], None], {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[1, [0, [0, 8, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}, None], 'and', [False, 2, [0, [0, 10, False], None], '""wyoming""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",what wyoming city has the largest population,"SELECT city_name FROM city WHERE population = ( SELECT MAX ( population ) FROM city WHERE state_name = ""wyoming"" ) AND state_name = ""wyoming"";","select city_name from city where population = ( select max ( population ) from city where state_name = ""wyoming"" ) and state_name = ""wyoming"";",what wyoming city has the largest population,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[what, wyoming, city, has, the, largest, population]","[what, wyoming, city, ha, the, largest, population]"
2,geo,"SELECT city_name FROM city WHERE popu

In [32]:
# Final pre-processed data with tokenised query and lemmatised question column
df1.columns

Index(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'clean_question', 'clean_query', 'query_lower', 'quest_lower', 'query_tokenised', 'quest_tokenised', 'quest_lemmatized'], dtype='object')

In [33]:
selected_columns_1 = df1[["query_tokenised","quest_lemmatized"]]
selected_columns_2 = df2[["query_tokenised","quest_lemmatized"]]
selected_columns_3 = df3[["query_tokenised","quest_lemmatized"]]

In [34]:
df_train = selected_columns_1.copy()
df_validate = selected_columns_2.copy()
df_test = selected_columns_3.copy()

In [35]:
df_train.head()

,query_tokenised,quest_lemmatized
0,"[select, count, (, *, ), from, head, where, age, >, 56]","[how, many, head, of, the, department, are, older, than, 56]"
1,"[select, name, ,, born_state, ,, age, from, head, order, by, age]","[list, the, name, born, state, and, age, of, the, head, of, department, ordered, by, age]"
2,"[select, creation, ,, name, ,, budget_in_billions, from, department]","[list, the, creation, year, name, and, budget, of, each, department]"
3,"[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]","[what, are, the, maximum, and, minimum, budget, of, the, department]"
4,"[select, avg, (, num_employees, ), from, department, where, ranking, between, 10, and, 15]","[what, is, the, average, number, of, employee, of, the, department, whose, rank, is, between, 10, and, 15]"


In [36]:
df_validate.head()

,query_tokenised,quest_lemmatized
0,"[select, count, (, *, ), from, singer]","[how, many, singer, do, we, have]"
1,"[select, count, (, *, ), from, singer]","[what, is, the, total, number, of, singer]"
2,"[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","[show, name, country, age, for, all, singer, ordered, by, age, from, the, oldest, to, the, youngest]"
3,"[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","[what, are, the, name, country, and, age, for, every, singer, in, descending, order, of, age]"
4,"[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, 'france, ']","[what, is, the, average, minimum, and, maximum, age, of, all, singer, from, france]"


In [37]:
df_test.head()

,query_tokenised,quest_lemmatized
0,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[what, is, the, biggest, city, in, wyoming]"
1,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[what, wyoming, city, ha, the, largest, population]"
2,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[what, is, the, largest, city, in, wyoming]"
3,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[where, is, the, most, populated, area, of, wyoming]"
4,"[select, city_name, from, city, where, population, =, (, select, max, (, population, ), from, city, where, state_name, =, ``, wyoming, '', ), and, state_name, =, ``, wyoming, '', ;]","[which, city, in, wyoming, ha, the, largest, population]"
